# Data Prep+Aug

In [1]:
import pandas as pd
import numpy as np
import csv
import cv2

In [2]:
sample = pd.read_csv('sample_data/driving_log.csv')
print('Number of sample examples: {}'.format(sample.shape[0]))

Number of sample examples: 8036


In [3]:
drive = pd.read_csv('data/track1/drive/driving_log.csv')
recov = pd.read_csv('data/track1/recovery/driving_log.csv')
print('Number of driving examples: {}'.format(drive.shape[0]))
print('Number of recovery examples: {}'.format(recov.shape[0]))

Number of driving examples: 7873
Number of recovery examples: 1457


In [4]:
# fourth attempt, using a generator

def get_batch(data, size=1024, path='sample_data', correction=0.2):
    
    n = data.shape[0]
    path = path + '/IMG/'
    
    for i in range(0, n, size):
        images = []
        measurements = []        
        j = min(n, i + size)
        
        # print(i, ' - ', j-1)       
        for _,row in data[i:j].iterrows():
            #img_center = cv2.imread(path + row[0].strip())
            img_center = cv2.imread(path + row[0].split('/')[-1])
            img_left = cv2.imread(path + row[1].split('/')[-1])
            img_right = cv2.imread(path + row[2].split('/')[-1])
            img = [img_center, img_left, img_right]

            steer_center = float(row[3])
            steer_left = steer_center + correction
            steer_right = steer_center - correction
            steer = [steer_center, steer_left, steer_right]

            # normal
            images.extend(img)
            measurements.extend(steer)    

            # flipped
            images.extend([cv2.flip(i,1) for i in img])
            measurements.extend([-1.0 * s for s in steer])  
            

        X = np.array(images)
        y = np.array(measurements)

        yield X,y

---
# Model Architecture

In [5]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Lambda
from keras.layers.convolutional import Convolution2D
from keras.layers import Cropping2D

#from keras.layers.pooling

Using TensorFlow backend.


In [6]:
model = Sequential()

# normalize
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160, 320, 3)))

# crop
model.add(Cropping2D(cropping=((70,25), (0,0))))

In [7]:
# nvidia architecture

model.add(Convolution2D(nb_filter=24,  nb_row=5, nb_col=5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(nb_filter=36,  nb_row=5, nb_col=5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(nb_filter=48,  nb_row=5, nb_col=5, subsample=(2,2), activation='relu'))

model.add(Convolution2D(nb_filter=64,  nb_row=3, nb_col=3, activation='relu'))
model.add(Convolution2D(nb_filter=64,  nb_row=3, nb_col=3, activation='relu'))

model.add(Flatten())

model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(1))

In [8]:
model.compile(loss='mse', optimizer='adam')

---

# Training

In [9]:
EPOCHS = 5
BATCH_SIZE = 2048

In [10]:
# batch mode

source = get_batch(sample, BATCH_SIZE, 'sample_data')
for X_train, y_train in source:
    model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=1, verbose=1)
        
for e in range(EPOCHS):

    print('*'*80)
    print('\nPass {}/{}\n'.format(e+1, EPOCHS))

    source = get_batch(drive, BATCH_SIZE, 'data/track1/drive')
    
    for X_train, y_train in source:
        model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=1, verbose=1)
        
    print('+'*80)  
    
    source = get_batch(recov, BATCH_SIZE, 'data/track1/recovery')
    
    for X_train, y_train in source:
        model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=1, verbose=1)
    
    print('\n'+('*'*80))

Train on 9830 samples, validate on 2458 samples
Epoch 1/1
9830/9830 [==============================] - 22s - loss: 0.0161 - val_loss: 0.0124
Train on 9830 samples, validate on 2458 samples
Epoch 1/1
9830/9830 [==============================] - 18s - loss: 0.0159 - val_loss: 0.0345
Train on 9830 samples, validate on 2458 samples
Epoch 1/1
9830/9830 [==============================] - 18s - loss: 0.0141 - val_loss: 0.0220
Train on 9081 samples, validate on 2271 samples
Epoch 1/1
9081/9081 [==============================] - 17s - loss: 0.0165 - val_loss: 0.0146
********************************************************************************
Pass 1/5

Train on 9830 samples, validate on 2458 samples
Epoch 1/1
9830/9830 [==============================] - 19s - loss: 0.0112 - val_loss: 0.0084
Train on 9830 samples, validate on 2458 samples
Epoch 1/1
9830/9830 [==============================] - 18s - loss: 0.0093 - val_loss: 0.0141
Train on 9830 samples, validate on 2458 samples
Epoch 1/1
9830/

In [11]:
model.save('models/model.h5')

---

# Archive

In [ ]:
# initial loop, no augmentation

"""
for line in lines[1:]:
    fn = line[0] #.split('/')[-1]
    img = cv2.imread('sample_data/'+fn)
    images.append(img)
    measurements.append(float(line[3]))    
"""
print()

In [ ]:
# second loopo, added flipping

"""
for line in lines[1:]:
    fn = line[0] #.split('/')[-1]
    img = cv2.imread('sample_data/'+fn)
    
    images.append(img)
    measurements.append(float(line[3]))    
        
    images.append(cv2.flip(img,1))
    measurements.append( -1.0 * float(line[3])) 
"""
print()

In [ ]:
# third loop, added multiple cameras
"""
CORRECTION = 0.2

images = []
measurements = []

with open('sample_data/driving_log.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader, None)
    c = 0
    for row in reader:
        img_center = cv2.imread('sample_data/' + row[0].strip())
        img_left = cv2.imread('sample_data/' + row[1].strip())
        img_right = cv2.imread('sample_data/' + row[2].strip())
        img = [img_center, img_left, img_right]
        
        steer_center = float(row[3])
        steer_left = steer_center + CORRECTION
        steer_right = steer_center - CORRECTION
        steer = [steer_center, steer_left, steer_right]

        # normal
        images.extend(img)
        measurements.extend(steer)    

        # flipped
        images.extend([cv2.flip(i,1) for i in img])
        measurements.extend([-1.0 * s for s in steer])   
        
        if c > 5000: break
        c += 1
        
X_train = np.array(images)
y_train = np.array(measurements)

"""
print()

In [ ]:
# dummy architecture
"""
model.add(Flatten())
model.add(Dense(1))
"""

In [ ]:
# full mode

# model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=EPOCHS, verbose=1)